In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/which-oil-is-the-best/Oil Well.csv')

In [ ]:
df.head()

**1.Data Cleaning**

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df=df.drop(index=[42])

In [ ]:
df.isnull().sum()

In [ ]:
df["retail markup"] = df["retail markup"].str.replace("%", "")
df["polyunsaturated (omega 3 & 6) %"] = df["polyunsaturated (omega 3 & 6) %"].str.replace("%", "")
df["monounsaturated %"] = df["monounsaturated %"].str.replace("%", "")
df["other fat %"] = df["other fat %"].str.replace("%", "")
df["saturated %"] = df["saturated %"].str.replace("%", "")
df["total reported fat content of oil %"] = df["total reported fat content of oil %"].str.replace("%", "")
df["other things (water, protein etc.), %"] = df["other things (water, protein etc.), %"].str.replace("%", "")
df["omega 3 %"] = df["omega 3 %"].str.replace("%", "")
df["omega 6 %"] = df["omega 6 %"].str.replace("%", "")
df["health rating"] = df["health rating"].str.replace("%", "")
df=df.fillna('others')
df["type"] = df["type"].str.replace("#REF!", "others")
df["omega 6 to omega 3 ratio: using calc"] = df["omega 6 to omega 3 ratio: using calc"].str.replace("no 3", "0")
df["trans fat"] = df["trans fat"].str.replace("%", "")
df=df.replace("-","0")
df=df.replace("-\n","0")
df['intl price per metric ton ($)']=df['intl price per metric ton ($)'].str.replace(",","")
df['omega 6 mg per 100g']=df['omega 6 mg per 100g'].str.replace(",","")
df['omega 3 mg per 100g']=df['omega 3 mg per 100g'].str.replace(",","")
df['yield, litres per hectare']=df['yield, litres per hectare'].str.replace(",","")

In [ ]:
df.dtypes

In [ ]:
df['taste strength index']=df['taste strength index'].astype(float)
df['intl price per metric ton ($)']=df['intl price per metric ton ($)'].astype(float)
df['UK retail cost per 100ml ($)']=df['UK retail cost per 100ml ($)'].astype(float)
df['retail markup']=df['retail markup'].astype(float)
df['saturated %']=df['saturated %'].astype(float)
df['polyunsaturated (omega 3 & 6) %']=df['polyunsaturated (omega 3 & 6) %'].astype(float)
df['monounsaturated %']=df['monounsaturated %'].astype(float)
df['other fat %x']=df['other fat %'].astype(float)
df['total reported fat content of oil %']=df['total reported fat content of oil %'].astype(float)
df['other things (water, protein etc.), %']=df['other things (water, protein etc.), %'].astype(float)
df['omega 6 to omega 3 ratio: using calc']=df['omega 6 to omega 3 ratio: using calc'].astype(float)
df['omega 6 mg per 100g']=df['omega 6 mg per 100g'].astype(float)
df['omega 3 mg per 100g']=df['omega 3 mg per 100g'].astype(float)
df['omega 3 %']=df['omega 3 %'].astype(float)
df['omega 6 %']=df['omega 6 %'].astype(float)
df['trans fat']=df['trans fat'].astype(float)
df['health rating']=df['health rating'].astype(float)
df['yield, litres per hectare']=df['yield, litres per hectare'].astype(float)
df['smoke point (C), average of sources']=df['smoke point (C), average of sources'].astype(float)
df['smoke point (F), average of sources']=df['smoke point (F), average of sources'].astype(float)
df['smoke point (C) source 1']=df['smoke point (C) source 1'].astype(float)
df['smoke point (C) source 2']=df['smoke point (C) source 2'].astype(float)
df['smoke point (C) source 3']=df['smoke point (C) source 3'].astype(float)
df['smoke point (C) source 4']=df['smoke point (C) source 4'].astype(float)
df['smoke point (C) source 2']=df['smoke point (C) source 2'].astype(float)

In [ ]:
df1=df.drop(['...based on flavour description','source','comment on omegas','note',
             'source names','links','omega 6 : omega 3 ratio (old)'], axis=1)

In [ ]:
df1=pd.get_dummies(df1)

**2.Correlation Analysis**

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(30, 20))
sns.heatmap(df1.corr())

**3.Taste Strength and Health Rating Analysis**

In [ ]:
sns.boxplot(x=df1['taste strength index'],y=df1['health rating'])

**4.Price, Cost and Markup Analysis**

In [ ]:
ax=sns.scatterplot(x=df1['intl price per metric ton ($)'],y=df1['UK retail cost per 100ml ($)'],
                   hue=df1['taste strength index'],size=df1['retail markup'],sizes=(10,500))
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

**5.Clustering and PCA**

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
sc = StandardScaler()
cl_sc=sc.fit_transform(df1)

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit(cl_sc)

In [ ]:
df1["clustering"] = clusters.labels_
df1.head()

In [ ]:
df1.groupby('clustering')['health rating'].mean().plot.bar()

In [ ]:
df1.groupby('clustering')['taste strength index'].mean().plot.bar()

In [ ]:
ax=sns.scatterplot(x=df1['intl price per metric ton ($)'],y=df1['UK retail cost per 100ml ($)'],
                   hue=df1['clustering'],size=df1['retail markup'],sizes=(10,500))
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

In [ ]:
pre = cl_sc
pca = PCA(n_components=2) 
pca.fit(pre)
pca_pre = pca.transform(pre)
pca_data = pd.DataFrame(pca_pre)

In [ ]:
for i in df1["clustering"].unique():
    tmp = pca_data.loc[df1["clustering"] == i]
    plt.scatter(tmp[0], tmp[1], label=i)
plt.legend()

**6.Description Analysis(NLP) by Clusters**

In [ ]:
df2=pd.DataFrame(df['...based on flavour description'])

In [ ]:
df2=df2.rename(columns={'...based on flavour description':'text'})

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import string

list_stopwords = set(stopwords.words('english'))

In [ ]:
df2['text'] = df2['text'].str.lower()
df2['text'] = df2['text'].apply(word_tokenize)
df2['text'] = df2['text'].apply(lambda x: [word for word in x if word not in list_stopwords])
df2['text'] = df2['text'].apply(lambda x : [word.translate(str.maketrans('', '', string.punctuation)) for word in x])
df2['text'] = df2['text'].apply(lambda x : [word for word in x if len(word) > 1])

In [ ]:
df2['taste strength index']=df1['taste strength index']
df2['health rating']=df1['health rating']
df2['clustering']=df1['clustering']

In [ ]:
df2

In [ ]:
df2_0=df2[df2['clustering']==0]
df2_1=df2[df2['clustering']==1]
df2_2=df2[df2['clustering']==2]
df2_3=df2[df2['clustering']==3]

In [ ]:
df3_0=df2_0['text'].explode()
df3_1=df2_1['text'].explode()
df3_2=df2_2['text'].explode()
df3_3=df2_3['text'].explode()

In [ ]:
df3_0=pd.DataFrame(df3_0)
df3_1=pd.DataFrame(df3_1)
df3_2=pd.DataFrame(df3_2)
df3_3=pd.DataFrame(df3_3)

In [ ]:
import squarify

<Words of people in clusster '0'>

In [ ]:
df3_0=pd.DataFrame(df3_0.groupby('text')['text'].count().sort_values(ascending=False).head(30))
df3_0=df3_0.rename(columns={'text': 'num'})

x = df3_0['num']
label = df3_0.index
squarify.plot(x, label=label,color=sns.color_palette('husl'))
plt.axis('off')
plt.show()

<Words of people in Cluster '1'>

In [ ]:
df3_1=pd.DataFrame(df3_1.groupby('text')['text'].count().sort_values(ascending=False).head(30))
df3_1=df3_1.rename(columns={'text': 'num'})

x = df3_1['num']
label = df3_1.index
squarify.plot(x, label=label,color=sns.color_palette('husl'))
plt.axis('off')
plt.show()

<Words of people in Cluster '2'>

In [ ]:
df3_2=pd.DataFrame(df3_2.groupby('text')['text'].count().sort_values(ascending=False).head(30))
df3_2=df3_2.rename(columns={'text': 'num'})

x = df3_2['num']
label = df3_2.index
squarify.plot(x, label=label,color=sns.color_palette('husl'))
plt.axis('off')
plt.show()

<Words of people in Cluster '3'>

In [ ]:
df3_3=pd.DataFrame(df3_3.groupby('text')['text'].count().sort_values(ascending=False).head(30))
df3_3=df3_3.rename(columns={'text': 'num'})

x = df3_3['num']
label = df3_3.index
squarify.plot(x, label=label,color=sns.color_palette('husl'))
plt.axis('off')
plt.show()